# Edwin Muiga Mwangi

### edwin.muiga@azubiafrica.org

### Week 11 Peer Programming Project

### Group 2 - Edison, Felix, Edwin


In [1]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
housing = pd.read_csv('housing.csv')

In [ ]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


# Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
train.shape, test.shape

((16512, 10), (4128, 10))

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
test_num = train.drop("ocean_proximity", axis=1)

In [ ]:
imputer.fit(test_num)

SimpleImputer(strategy='median')

In [ ]:
imputer.statistics_

array([-1.1851e+02,  3.4260e+01,  2.9000e+01,  2.1290e+03,  4.3700e+02,
        1.1670e+03,  4.1000e+02,  3.5458e+00,  1.7985e+05])

In [ ]:
test_num.median().values

array([-1.1851e+02,  3.4260e+01,  2.9000e+01,  2.1290e+03,  4.3700e+02,
        1.1670e+03,  4.1000e+02,  3.5458e+00,  1.7985e+05])

In [ ]:
Y = imputer.transform(test_num)

In [ ]:
test_tr = pd.DataFrame(Y, columns=test_num.columns)

In [ ]:
test_cat = train[["ocean_proximity"]]

test_cat.head(10)

,ocean_proximity
14196,NEAR OCEAN
8267,NEAR OCEAN
17445,NEAR OCEAN
14265,NEAR OCEAN
2271,INLAND
17848,<1H OCEAN
6252,<1H OCEAN
9389,NEAR BAY
6113,<1H OCEAN
6061,<1H OCEAN


In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
test_cat_1hot = cat_encoder.fit_transform(test_cat)
test_cat_1hot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

In [ ]:
test_cat_1hot.toarray()

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [ ]:
cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [ ]:
test["rooms_per_household"] = test["total_rooms"]/test["households"]
test["bedrooms_per_room"] = test["total_bedrooms"]/test["total_rooms"]
test["population_per_household"]=test["population"]/test["households"]

In [ ]:
corr_matrix = test.corr()

corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value          1.000000
median_income               0.677502
total_rooms                 0.134697
rooms_per_household         0.130928
housing_median_age          0.113585
households                  0.074249
total_bedrooms              0.056667
population                 -0.019003
longitude                  -0.044062
population_per_household   -0.121853
latitude                   -0.149295
bedrooms_per_room          -0.249196
Name: median_house_value, dtype: float64

In [ ]:
test_labels = test['median_house_value'].copy()

# drop the labels from the train set via the pandas drop function:

test = test.drop('median_house_value', axis=1) # axis=1 means that median_house_value should be dropped column wise, meaning, the whole column will be dropped

test_num = test.drop("ocean_proximity", axis=1) # re-do since the additional columns rooms_per_household, bedrooms_per_room and population_per_household where added

test_labels.head()

20046     47700.0
3024      45800.0
15663    500001.0
20484    218600.0
9814     278000.0
Name: median_house_value, dtype: float64

In [ ]:
test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,INLAND,4.192201,NaN,3.877437
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,INLAND,5.039384,NaN,2.679795
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,NEAR BAY,3.977155,NaN,1.360332
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,<1H OCEAN,6.163636,NaN,3.444444
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,NEAR OCEAN,5.492991,NaN,2.483645


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([('imputer',SimpleImputer(strategy="median")),('std_scaler', StandardScaler())])

test_num_tr = num_pipeline.fit_transform(test_num)

In [ ]:
from sklearn.compose import ColumnTransformer
num_attribs = list(test_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs), ("cat",OneHotEncoder(), cat_attribs) ])
test_prepared = full_pipeline.fit_transform(test)

## Modelling

In [ ]:
test_prepared.shape

(4128, 16)

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(test_prepared, test_labels)

LinearRegression()

In [ ]:
some_data = test.iloc[:5]
some_labels = test_labels.iloc[:5]

# transform / prepare some data
some_data_prepared = full_pipeline.transform(some_data)

# make predictions
print("Predictions:", lin_reg.predict(some_data_prepared))

Predictions: [ 44716.39774671 144106.84827943 307690.99596094 279321.21705505
 268801.9121698 ]


In [ ]:
print("Labels:", list(some_labels))

Labels: [47700.0, 45800.0, 500001.0, 218600.0, 278000.0]


In [ ]:
from sklearn.metrics import mean_squared_error

test_predictions = lin_reg.predict(test_prepared)
lin_mse = mean_squared_error(test_labels, test_predictions)
lin_rmse = np.sqrt(lin_mse)

lin_rmse

68748.60903173663

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(test_prepared, test_labels)

DecisionTreeRegressor()

In [ ]:
test_predictions = tree_reg.predict(test_prepared)
tree_mse = mean_squared_error(test_labels, test_predictions)
tree_rmse = np.sqrt(tree_mse)

tree_rmse

0.0

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, test_prepared, test_labels,scoring="neg_mean_squared_error", cv=10)

tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
display_scores(tree_rmse_scores)

Scores: [84059.21328198 77424.83195625 79326.94017073 81897.54243972
 81817.44843157 76148.96154162 73527.79474482 81022.64244589
 77297.4793611  76684.54775974]
Mean: 78920.74021334219
Standard deviation: 3080.2444497689135


In [ ]:
lin_scores = cross_val_score(lin_reg, test_prepared,test_labels,scoring="neg_mean_squared_error", cv=10)

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [67404.97038787 69127.6550799  75864.82154489 72152.56108756
 72250.30236857 64321.316356   67504.69067265 74197.44724277
 68214.96737372 71915.32682217]
Mean: 70295.40589361063
Standard deviation: 3371.4007163622096


In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(test_prepared, test_labels)

test_predictions = forest_reg.predict(test_prepared)
forest_mse = mean_squared_error(test_labels, test_predictions)
forest_rmse = np.sqrt(tree_mse)

forest_rmse

0.0

In [ ]:
forest_scores = cross_val_score(forest_reg, test_prepared,test_labels,scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [54114.2265647  52690.98647853 57691.99424027 61821.94900048
 53932.26197141 52187.2833516  52325.58767717 57604.54969742
 58211.67541133 53441.4878003 ]
Mean: 55402.200219320206
Standard deviation: 3066.1984292715897


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [ {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid,cv=5,scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(test_prepared, test_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
grid_search.best_params_ # our result {'max_features': 8, 'n_estimators': 30}

# Since 8 and 30 are the maximum values that were evaluated, you should
#probably try searching again with higher values, since the score may
#continue to improve.

{'max_features': 4, 'n_estimators': 30}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(max_features=4, n_estimators=30)

In [ ]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

70922.93028812134 {'max_features': 2, 'n_estimators': 3}
61341.608650706694 {'max_features': 2, 'n_estimators': 10}
58457.19050456207 {'max_features': 2, 'n_estimators': 30}
68489.70349675733 {'max_features': 4, 'n_estimators': 3}
58987.350176943844 {'max_features': 4, 'n_estimators': 10}
55414.69095447628 {'max_features': 4, 'n_estimators': 30}
64080.93062369513 {'max_features': 6, 'n_estimators': 3}
58312.319675515384 {'max_features': 6, 'n_estimators': 10}
55946.852353501294 {'max_features': 6, 'n_estimators': 30}
65372.44835302116 {'max_features': 8, 'n_estimators': 3}
57311.40765102424 {'max_features': 8, 'n_estimators': 10}
55796.24807143981 {'max_features': 8, 'n_estimators': 30}
69119.6710119884 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
60193.80764907009 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
66050.64309727296 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
58507.20632111863 {'bootstrap': False, 'max_features': 3, 'n_estimators':

In [ ]:
import pickle
#from sklearn.externals import joblib

filename = 'forest_housing_model.pkl'

#joblib.dump(grid_search.best_estimator_,filename)

filename = 'forest_housing_model.sav'
pickle.dump(grid_search.best_estimator_, open(filename, 'wb'))

In [ ]:
model = pickle.load(open(filename, 'rb'))